# Assess the structural overlap of actual pairs vs tranlsations

In [1]:
from nomelt.thermo_estimation import ESMFoldDGEstimator

[2023-08-07 10:13:51,975] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-08-07 10:14:00.670851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 10:14:00.985368: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import datasets
from nomelt.translate import translate_sequences
import torch
import esm

import Bio.pairwise2 as pairwise2
import biotite.structure.io as bsio

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
from Bio.PDB import *
import nglview as nv
import ipywidgets

In [4]:
from Bio.Align import substitution_matrices

In [5]:
import datasets
import numpy

In [6]:
mat = substitution_matrices.load('BLOSUM62')

In [7]:
parser = PDBParser()

In [8]:
torch.cuda.is_available()

True

## translate a training example

In [9]:
dataset = datasets.load_from_disk('../data/dataset')

In [25]:
train_example = dataset['train'].select([numpy.random.randint(len(dataset['train']))])[0]

In [26]:
meso=train_example['meso_seq']
thermo=train_example['thermo_seq']

In [27]:
translation = translate_sequences([meso], model_path='../data/nomelt-model/model')[0]

In [28]:
translation

'MARRSKRMRQIRERIDRTRQYPVEEAIQLLKELSSVKFDESVDVAVNLGVDPRKSDQVVRGATVLPHGTGAKVRVAVFAQGAAAEAAREAGADIVGFEDLAERIKGGELDFDVVIATPDAMRVVGQLGKILGPRGLMPNPKVGTVTADVAQAVRNAKAGQVRYRTDKAGIIHCTIGKVSFEPKALKENLEALLGDLQKAKPSAAKGTYMKKITLSTTMGPGLAVDRASLSV'

### true alignment

In [33]:
alignment = pairwise2.align.globalds(meso, thermo, mat, -2,-1, one_alignment_only=True, penalize_extend_when_opening=False, penalize_end_gaps=False)

In [34]:
print(pairwise2.format_alignment(*alignment[0], full_sequences=True))

MAKQSKKLTAAY----KAFDREKSFALVEAVKMV-R-ANATAKFDETVEISMNLGIDPRHADQMVRG-MVGLPNGTGKTVRVAVFARGAKAEEAQAAGADIVGADDLAAAVQDGK---MDFDRCIATPDMMGLVGRLGKVLGPRGLMPNPKLGTVTMDV-KGAIAAARAGQVEYRAEKAGIVHAGIGKVSFDEDKLIENARAFV-DAIQKARPTGAKGTYVKKVAVSSSMGVGVKVDVASL-SANG
| |..|.    |    |..|........||...| . ||  ||||||......||.|||||||.||| .. ||.||||||.|.|||.|.||.||.|||||.|||..|....|  |   .|.|..||||||||.||||||.|||.|||||||.||||..| | |...|.||..|||..||||.|..|||.||...||.||.|. | |||.|..|..|||.|.|.|..||.||.||||..|.. .   
M-KRGKR----YLENLKLYDKTQQYSSDEAMDIVLKTAN--AKFDETIDLAVRLGVDPRHADQQVRGTVI-LPHGTGKTVKVLVFAKGEKAKEAEAAGADYVGAEELVEKIQ--KENWFDYDVVIATPDMMGVVGRLGKLLGPKGLMPNPKSGTVTFEVEK-AVKEAKAGKIEYRIDKAGIIHVPIGKKSFGKEKLLENFRT-VMDAIIKSKPAAAKGQYIKSVVLSSTMGPGVKVNPARIFE---
  Score=683



In [35]:
alignment = pairwise2.align.globalds(meso, translation, mat, -2,-1, one_alignment_only=True, penalize_extend_when_opening=False, penalize_end_gaps=False)

In [36]:
print(pairwise2.format_alignment(*alignment[0], full_sequences=True))

MAKQSKKLTAAYKAFDREKSFALVE-AVKMVRAN-ATAKFDETVEISMNLGIDPRHADQMVRGM-VGLPNGTG-KTVRVAVFARGAKAEEAQAAGADIVGADDLAAAVQDGKMDFDRCIATPDMMGLVGRLGKVLGPRGLMPNPKLGTVTMDVKGAIAAARAGQVEYRAEKAGIVHAGIGKVSFDEDKLI-ENARAFV-DAIQKARPTGAKGTYVKKVAVSSSMGVGVKVDVASLSANG
||..||.........||..... || |..... . ...||||.|....|||.|||..||.|||. | ||.||| | |||||||.||.||.|..|||||||..|||.....|..|||..|||||.|..||.|||.|||||||||||.||||.||..|...|.||||.||..||||.|..|||||| |.|.. ||..|.. | .|||.|..|||||.||...|..||.|..||.||||.  
MARRSKRMRQIRERIDRTRQYP-VEEAIQLLK-ELSSVKFDESVDVAVNLGVDPRKSDQVVRGATV-LPHGTGAK-VRVAVFAQGAAAEAAREAGADIVGFEDLAERIKGGELDFDVVIATPDAMRVVGQLGKILGPRGLMPNPKVGTVTADVAQAVRNAKAGQVRYRTDKAGIIHCTIGKVSF-EPKALKENLEALLGD-LQKAKPSAAKGTYMKKITLSTTMGPGLAVDRASLSV--
  Score=684



#### energy differences

In [37]:
estimator = ESMFoldDGEstimator(sequences=[meso, thermo, translation], ids=['meso', 'thermo', 'trans'])

In [38]:
estimator.run()

Running ESMfold in 3 proteins.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.97s/it]


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.690416 seconds.
core.import_pose.import_pose: File './tmp/tmp7m3kb7qe/meso.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLY:CtermProteinFull 233
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database fi

{'meso': -618.2627009136091,
 'thermo': -672.0713673963103,
 'trans': -642.3347080698281}

The translation is stabilizing, but does not do this for every sequence.

### structure differences

In [40]:
meso_struct = parser.get_structure('train_meso', estimator.pdb_files_dict['meso'])

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1700
  warnings.warn(


In [41]:
nv.show_biopython(meso_struct, gui=True)

NGLWidget()

In [42]:
thermo_struct = parser.get_structure('train_thermo', estimator.pdb_files_dict['thermo'])
nv.show_biopython(thermo_struct, gui=True)

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'PARENT' at line 1782
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1784
  warnings.warn(


NGLWidget()

In [44]:
trans_struct = parser.get_structure('train_translation', estimator.pdb_files_dict['trans'])
nv.show_biopython(trans_struct, gui=True)

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'PARENT' at line 1736
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1738
  warnings.warn(


NGLWidget()

The structure is mostly recovered for the training example. 

## translate 1ENH as a check, no ground truth to compare

### Get translations

In [9]:
ENH1 = "RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI"

In [10]:
ENH1_translation = translate_sequences([ENH1], model_path='../data/nomelt-model/model')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [11]:
ENH1_translation

['RPRTAFTAEQLQRLKAEFQRSRYLSEQRRQELARELGLNESQIKIWFQNKRAKIKKASGVK']

In [12]:
alignment = pairwise2.align.globalds(ENH1, ENH1_translation[0], mat, -2,-1, one_alignment_only=True, penalize_extend_when_opening=False, penalize_end_gaps=False)

In [13]:
print(pairwise2.format_alignment(*alignment[0], full_sequences=True))

RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI-------
||||||..|||.|||.||...|||.|.|||.|..||||||.|||||||||||||       
RPRTAFTAEQLQRLKAEFQRSRYLSEQRRQELARELGLNESQIKIWFQNKRAKIKKASGVK
  Score=216



In [10]:
estimator = ESMFoldDGEstimator()

In [11]:
estimator.run(sequences=[ENH1, "RPRTAFTAEQLQRLKAEFQRSRYLSEQRRQELARELGLNESQIKIWFQNKRAKIKKASGVK"], ids=['enh1_wt', 'enh1_trans'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.81s/it]


PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1673372157 seed_offset=0 real_seed=1673372157
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1673372157 RG_type=mt19937
core.che

{'enh1_wt': -133.4652284624724, 'enh1_trans': -162.53537554680292}

Apparently the translation is much more stable.

In [13]:
trans = parser.get_structure('1enh_t', estimator.pdb_files_history['enh1_trans'])

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 519
  warnings.warn(


In [14]:
nv.show_biopython(trans, gui=True)

NGLWidget()

The model extended the crap out of 1ENH but it actually has a similar motif... if has an additional floppy helix. I wonder if it would be more stable and still be a transcription factor?

In [54]:
wt = parser.get_structure('1enh_wt', estimator.pdb_files_dict['enh1_wt'])

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'PARENT' at line 469
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 471
  warnings.warn(


In [55]:
nv.show_biopython(wt, gui=True)

NGLWidget()

### cut off the tail and see what happens.

In [67]:
tail_length = len(alignment[0].seqA) - len(alignment[0].seqA.rstrip('-'))
tail_length

30

In [81]:
ENH1_translation_cut = ENH1_translation[0][2:-tail_length]
ENH1_translation_cut

'KPRTSFTNEQIARLEKEFNRQRYLSAQRRAELAASLNLTETQVKIWFQNRRYKT'

In [82]:
estimator = ESMFoldDGEstimator(sequences=[ENH1, ENH1_translation_cut], ids=['enh1_wt', 'enh1_trans_cut'])

In [83]:
estimator.run()

Running ESMfold in 2 proteins.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.678758 seconds.
core.import_pose.import_pose: File './tmp/tmpfd_6o41h/enh1_wt.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ILE:CtermProteinFull 54
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database 

{'enh1_wt': -130.82074528039993, 'enh1_trans_cut': -138.2588679216842}

In [84]:
trans = parser.get_structure('1enh_t_cut', estimator.pdb_files_dict['enh1_trans_cut'])

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 468
  warnings.warn(


In [86]:
estimator.pdb_files_dict

{'enh1_wt': './tmp/tmpfd_6o41h/enh1_wt.pdb',
 'enh1_trans_cut': './tmp/tmpfd_6o41h/enh1_trans_cut.pdb'}

In [85]:
nv.show_biopython(trans, gui=True)

NGLWidget()

Even when we cut off the tails, the energy is slightly better. this is good.